In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

In [3]:
re = pd.read_pickle(r'L:\orderLog\ret\orderLogWithRet_20200921.pkl')
re[re['indexCat'].isnull()]['secid'].unique()

array([1688536., 1688127.])

In [20]:
read_memb_daily(db1, 'index_memb', int(startDate), int(startDate), skey=[1601099])

,date,weight,skey,index_id,index_name
0,20200921,0.527051,1601099,1000905,IC
1,20200921,0.540000,1601099,3011046,AMAC 金融


In [14]:
import os
import glob
import datetime
import numpy as np
import pandas as pd

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20200921'
endDate = '20201028'

readPath = r'A:\orderLog\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

indexCatData = read_memb_daily(db1, 'index_memb', int(startDate), int(endDate), index_id=[1000300, 100905, 1000852, 1000985])
indexCatData = indexCatData[['index_id', 'skey', 'date']].reset_index(drop=True)
indexCatData = indexCatData.rename(columns={'index_id':'indexCat', 'skey':'secid'})
indexCatData.loc[indexCatData['indexCat'] == 1000985, 'indexCat'] = 1000852
indexCatData['ID'] = indexCatData['secid']

# addData = pd.DataFrame({'indexCat': [1000300, 1000905, 1000852], 'ID': [1000300, 1000905, 1000852]})
# indexCatData = pd.concat([indexCatData, addData], sort=False).reset_index(drop=True)

d1 = pd.read_pickle(r'E:\stockBeta_L_IC_60d.pkl').reset_index()
d1['indexCat'] = 1000905
d1 = d1[(d1['index'] >= int(startDate)) & (d1['index'] <= int(endDate))]
d2 = pd.read_pickle(r'E:\stockBeta_L_IF_60d.pkl').reset_index()
d2['indexCat'] = 1000300
d2 = d2[(d2['index'] >= int(startDate)) & (d2['index'] <= int(endDate))]
d3 = pd.read_pickle(r'E:\stockBeta_L_CSI_60d.pkl').reset_index()
d3['indexCat'] = 1000852
d3 = d3[(d3['index'] >= int(startDate)) & (d3['index'] <= int(endDate))]
d = pd.concat([d1, d2, d3]).sort_values(by='index')
betaData = []
for i in d.columns[:-2]:
    col = [i] + ['index', 'indexCat']
    re = d[col]
    re = re.rename(columns={i: 'beta_60', 'index':'date'})
    re['secid'] = int(i[2:]) + 2000000 if i[:2] == 'SZ' else int(i[2:]) + 1000000
    betaData += [re]
betaData = pd.concat(betaData).reset_index(drop=True)

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [13]:
import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz

def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

def build_query(start_date=None, end_date=None, index_id=None):
    query = {}
    def parse_date(x):
        if type(x) == int:
            return x
        elif type(x) == str:
            if len(x) != 8:
                raise Exception("date must be YYYYMMDD format")
            return int(x)
        elif type(x) == datetime.datetime or type(x) == datetime.date:
            return x.strftime("%Y%m%d").astype(int)
        else:
            raise Exception("invalid date type: " + str(type(x)))
    if start_date is not None or end_date is not None:
        query['date'] = {}
        if start_date is not None:
            query['date']['$gte'] = parse_date(start_date)
        if end_date is not None:
            query['date']['$lte'] = parse_date(end_date)
    def parse_symbol(x):
        if type(x) == int:
            return x
        else:
            return int(x)
    if index_id:
        if type(index_id) == list or type(index_id) == tuple:
            query['index_id'] = {'$in': [parse_symbol(x) for x in index_id]}
        else:
            query['index_id'] = parse_symbol(index_id)
    return query

def write_memb_data(db, name, df):
    collection = db[name]
    df1 = []
    for symbol in df['index_id'].unique():
        if symbol in collection.distinct('index_id'):
            symbol = int(symbol)
            m_ax = pd.DataFrame.from_records(collection.find({'index_id':{'$in':[symbol]}}).sort([('date',-1)]).skip(0).limit(1))['date'].values[0]
            df2 = df[(df['index_id'] == symbol) & (df['date'] > m_ax)]
            print(df2)
            df1 += [df2]
        else:
            print(symbol)
            df2 = df[(df['index_id'] == symbol)]
            print(df2)
            df1 += [df2]
    df1 = pd.concat(df1).reset_index(drop=True)
    df1 = df1.to_dict('records')
    collection.insert_many(df1) 

def delete_memb_data(db, name, start_date=None, end_date=None, index_id=None):
    collection = db[name]
    query = build_query(start_date, end_date, index_id)
    if not query:
        print('cannot delete the whole table')
        return None
    collection.delete_many(query)  


database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)

In [8]:
re[re['secid'] == 1601099]['indexCat'].unique()

array([nan])

In [2]:
rawOrderLog = pd.read_pickle(r'A:\temp\zs66 speed\speedCompare_zs66.pkl')
rawOrderLog = rawOrderLog.fillna(0)
for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]


targetStock = rawOrderLog['secid'].unique()
targetStock = np.array([int(str(i)[1:]) for i in targetStock])
targetStockSZ = sorted(targetStock[targetStock < 600000])
targetStockSH = sorted(targetStock[targetStock >= 600000])

rawOrderLog['clock'] = rawOrderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
rawOrderLog['broker'] = rawOrderLog['accCode'] // 100
rawOrderLog["broker"] = np.where(rawOrderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), rawOrderLog['accCode'] // 10000, rawOrderLog["broker"])
rawOrderLog['colo_broker'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['broker'].astype('str')
rawOrderLog['order'] = rawOrderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
rawOrderLog['group'] = rawOrderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
rawOrderLog['startClock'] = rawOrderLog.groupby(['order'])['clockAtArrival'].transform('first')
rawOrderLog['duration'] = rawOrderLog['clockAtArrival'] - rawOrderLog['startClock']
rawOrderLog['orderPrice'] = rawOrderLog['orderPrice'].apply(lambda x: round(x, 2))
rawOrderLog['tradePrice'] = rawOrderLog['tradePrice'].apply(lambda x: round(x, 2))
rawOrderLog['orderDirection1'] = np.where(rawOrderLog["orderDirection"] == -2, -1, np.where(
    rawOrderLog["orderDirection"] == 2, 1, rawOrderLog["orderDirection"]))
orderLog = rawOrderLog.copy()

### make sure updateType == 7 orders < 20 per account, < 100 in total
assert(orderLog[orderLog['updateType'] == 7].groupby('accCode')['vai'].count().max() < 20)
assert(orderLog[orderLog['updateType'] == 7].groupby('accCode')['vai'].count().sum() < 100)

### make sure no order has shares > 80w or notional > 800w
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
# assert(orderLog[orderLog['absOrderSize'] > 800000].shape[0] == 0)
# assert(orderLog[orderLog['orderNtl'] > 8000000].shape[0] == 0)
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

### make sure same direction in same colo_broker
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(orderLog[orderLog['directNum'] != 1][['date', 'accCode', 'secid', 'vai', 'orderDirection']])
    orderLog = orderLog[orderLog['directNum'] == 1]

assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

## make sure each account, secid, vai only has one insertion
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))

### make sure there is no unexpected updateType 
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 2, 4), (0, 2, 2, 1, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 4, 1, 4), (0, 2, 2, 4, 1, 4), (0, 4, 2, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 4, 1, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3), (0, 4, 2, 1, 3), (0, 2, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 2, 4, 1, 4, 3)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3)]), 2,
                     np.where(checkLog['updateType'].isin([(0, 3)]), 3,
                     np.where(checkLog['updateType'].isin([(0,), (0, 2), (0, 2, 1), (0, 2, 2), (0, 2, 4, 1)]), 4, 5)))))
display(set(checkLog["updateType"].unique()) - set([(0, 2, 4), (0, 2, 2, 4), (0, 2, 2, 1, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4),
                                                    (0, 2, 4, 1, 4), (0, 4), (0, 4, 1, 4), (0, 2, 2, 4, 1, 4), (0, 4, 2, 4), 
                                                    (0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3),
                                                    (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 4, 1, 3), (0, 2, 2, 1, 4, 3),
                                                    (0, 4, 2, 4, 1, 3), (0, 4, 2, 1, 3), (0, 2, 1, 3), (0, 2, 2, 1, 3),
                                                    (0, 3), (0,), (0, 2), (0, 2, 1), (0, 2, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 2, 4, 1, 4, 3), 
                                                    (0, 2, 2), (0, 2, 4, 1)]))

display(checkLog[checkLog['status'] == 5])
orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### check status==0 got all traded
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### check status==1 got partial traded
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### check if any cancellation within 1 sec
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog["server"] = orderLog["colo"].apply(lambda x: x.split("_")[0] + x.split("_")[1] + x.split("_")[2])
orderLog["server_account"] = orderLog["server"] + '_' + orderLog['accCode'].astype('str')
orderLog["colo_account"] = orderLog["colo"].str[:2] + '_' + orderLog['accCode'].astype('str')
orderLog["time"] = orderLog["clock"].apply(lambda x: x.strftime("%H%M%S")).astype(int)

In [102]:
orderLog['order'] = orderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog["server"] = orderLog["colo"].apply(lambda x: x.split("_")[0] + x.split("_")[1] + x.split("_")[2])
orderLog["server_account"] = orderLog["server"] + '_' + orderLog['accCode'].astype('str')
checkLog = orderLog[orderLog['updateType'].isin([0, 3, 4])].reset_index(drop=True)
checkLog = checkLog[checkLog['duration'] <= 1e6]

ol = checkLog[(checkLog["updateType"] == 0) & (checkLog["ars"] == 11)]["order"].unique()
checkLog = checkLog[checkLog["order"].isin(ol)]


checkLog['accountNum'] = checkLog.groupby(['date', 'secid', 'vai'])['server_account'].transform('nunique')
checkLog = checkLog[checkLog['accountNum'] >= 2]
checkLog['cumFillSize'] = checkLog.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
checkLog['status'] = np.where(checkLog['cumFillSize'] == 0, 2, 
                                np.where(checkLog['cumFillSize'] < checkLog['absOrderSize'], 1, 0))
checkLog = checkLog.groupby(['group', 'order'])[['date', 'server_account', 'secid', 'vai', 'status']].first().reset_index()
exchangeLog = checkLog[checkLog['secid'] >= 2000000]

for exchg in ['SZE']:
    
    print(exchg)
    
    if exchg == 'SSE':
        exchangeLog = checkLog[checkLog['secid'] < 2000000]
    else:
        exchangeLog = checkLog[checkLog['secid'] >= 2000000]
    
    result = {}
    for col in ['date', 'broker1', 'broker2', 'isFast']:
        result[col] = []

    for group, groupData in exchangeLog.groupby(['group']):
        date = groupData['date'].values[0]
        brokerLs = groupData['server_account'].values
        if len(brokerLs) == 0:
            continue
        statusLs = groupData['status'].values
        ixLs = [i for i in range(len(brokerLs))]
        for k, broker1, status1 in zip(ixLs[:-1], brokerLs[:-1], statusLs[:-1]):
            for broker2, status2 in zip(brokerLs[k+1:], statusLs[k+1:]):
                if broker1 != broker2:
                    result['date'] += [date, date]
                    result['broker1'] += [broker1, broker2]
                    result['broker2'] += [broker2, broker1]

                    if status1 < status2:
                        result['isFast'] += [1, 0]
                    elif status1 > status2:
                        result['isFast'] += [0, 1]
                    else:
                        result['isFast'] += [-1, -1]                

    result = pd.DataFrame(result)
    result = result[result['isFast'] != -1]
    result['count'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('count')
    result['fasterSum'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
    result['faster'] = result['fasterSum']/result['count']

    
#     summary = pd.DataFrame()
#     summaryCols = []
#     countCols = []
#     fasterCols = []
#     for date in result['date'].unique():
#         a = result[result['date'] == date].reset_index(drop=True)
#         a['count'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('count')
#         a['fasterSum'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
#         a['faster'] = a['fasterSum']/a['count']
#         a = a.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
#         a.columns = ['broker1', 'broker2', '%s count'%date, '%s faster'%date]
#         summaryCols += ['%s count'%date, '%s faster'%date]
#         countCols += ['%s count'%date]
#         fasterCols += ['%s faster'%date]
        
#         if summary.empty:
#             summary = a.copy()
#         else:
#             summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')
    
#     a = result.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
#     a.columns = ['broker1', 'broker2', 'total count', 'total faster']
#     summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')
#     summaryCols = ['total count', 'total faster'] + summaryCols
    
#     countCols += ['total count']
#     fasterCols += ['total faster']
#     summary['b1'] = summary['broker1'].str[7:11].astype(int) // 100
#     summary['b2'] = summary['broker2'].str[7:11].astype(int) // 100
# #     summary = summary.groupby(['broker1', 'broker2'])[summaryCols].first()
#     for col in countCols:
#         summary[col] = summary[col].fillna(0)
#         summary[col] = summary[col].astype('int64')
        
#     for col in fasterCols:
#         summary[col] = summary[col].fillna(-1)
#         summary[col] = summary[col].apply(lambda x: '%.0f%%'%(x*100))
#         summary[col] = np.where(summary[col] == '-100%', ' ', summary[col])
    
# #     summary = summary.rename(columns={"broker1":"account1", "broker2":"account2"})
# #     summary = summary.groupby(['account1', 'account2'])[summaryCols].first()
# #     from IPython.display import display, HTML
# #     display(HTML(summary.iloc[:, :2].to_html()))
#     summary["tt"]=summary[["broker1", "broker2"]].min(axis=1).astype(str) + ' - ' + summary[["broker1", "broker2"]].max(axis=1).astype(str)
#     summary = summary.sort_values(by=["tt", "broker1"])
#     s1 = summary.groupby("tt").first().reset_index(drop=True)
#     display(s1)

SZE


In [112]:
startDate = '20200722'
endDate = '20200724'
readPath = r'L:\orderLog\result\marketPos'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'SZspeed_***.pkl')))
dateLs = np.array([os.path.basename(i).split('.')[0].split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
checkData = []
for path in dataPathLs:
    data = pd.read_pickle(path)
    checkData += [data]
checkLog = pd.concat(checkData).reset_index(drop=True)   
checkLog = checkLog[checkLog['ars'] == 11]
display(checkLog.shape[0])
display(checkLog[checkLog.duplicated(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep='last')].shape[0])
display(checkLog[checkLog.duplicated(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)].shape[0])
display(checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep='last').shape[0])
display(checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False).shape[0])
checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog['kk'] = checkLog.groupby(['date', 'ApplSeqNum'])['accCode'].transform('nunique')
checkLog = checkLog[checkLog['kk'] != 1]
# checkLog = checkLog[(checkLog['date'] != 20200724) | ((checkLog['date'] == 20200724) & (checkLog['TransactTime'] <= 93300000))]
checkLog['time_diff'] = checkLog['clockAtArrival'] - checkLog['start_time']
checkLog['colo1'] = checkLog['colo'].str[:2]
checkLog['colo_broker'] = checkLog['colo1'] + '_' + checkLog["accCode"].astype(str).str[:2]

from IPython.display import display, HTML
re1 = checkLog.groupby(['date', 'colo', 'accCode'])['time_diff'].describe().fillna(0).astype(int).reset_index()
# re1 = re1[re1['count'] > 20].reset_index()
re1['accCode'] = re1['accCode'].astype(int)
display(HTML(re1[['date', 'colo', 'accCode', 'count', '50%']].groupby(['date', 'colo', 'accCode']).first().to_html()))

5564

1801

2603

3763

2961

In [111]:
re1

count    mean     std     min     25%     50%  \
date     colo     accCode                                                  
20200722 zs_94_04 6237.0      16  271966  296581    3267   51310  143180   
         zs_96_08 6282.0      16   93729   93495   60926   62392   64923   
20200723 zs_94_04 6237.0      31  146245  124197   30555   41902   85503   
         zs_96_08 6282.0      31   84605   38210   57249   62862   76534   
20200724 zs_94_04 6237.0       9  412283  228870  115876  195178  494082   
         zs_96_08 6282.0       9  110932   59156   61474   80480   86206   

                              75%     max  
date     colo     accCode                  
20200722 zs_94_04 6237.0   353781  926561  
         zs_96_08 6282.0    77933  441124  
20200723 zs_94_04 6237.0   213189  481457  
         zs_96_08 6282.0    90492  265392  
20200724 zs_94_04 6237.0   574295  766774  
         zs_96_08 6282.0   102661  219258

In [107]:
checkLog['kk'] = checkLog.groupby(['date', 'ApplSeqNum'])['accCode'].transform('nunique')
checkLog[checkLog['kk'] != 1]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,agg,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,start_time,end_time,statusLs,TradePriceLs,TradeQtyLs,vai,ars,exchange,group,order,colo,accCode,kk
906,28,5801753.0,1595382873482178,32729272,1.0,1200.0,483800.0,684504.0,1.0,95432890,0,0,0,2000028,2.805802e+09,20200722,1.595383e+15,1.595383e+15,"(0, 4)","(0, 483800)","(0, 1200)",680504.0,11.0,SZE,1453.0,1080.0,zs_94_04,6237.0,2.0
907,28,5801753.0,1595382873495114,32729560,1.0,300.0,483800.0,684604.0,1.0,95432900,0,0,0,2000028,2.805802e+09,20200722,1.595383e+15,1.595383e+15,"(0, 4, 3)","(0, 483800, 0)","(0, 100, 0)",680504.0,11.0,SZE,1453.0,3133.0,zs_96_08,6282.0,2.0
6708,729,1079350.0,1595381460395224,5331461,1.0,3000.0,74400.0,389813.0,0.0,93059800,0,0,0,2000729,7.290108e+10,20200722,1.595381e+15,1.595381e+15,"(0, 4)","(0, 74400)","(0, 3000)",378613.0,11.0,SZE,1802.0,1308.0,zs_94_04,6237.0,2.0
6711,729,1079350.0,1595381460421449,5331997,1.0,400.0,74400.0,389813.0,0.0,93059830,0,0,0,2000729,7.290108e+10,20200722,1.595381e+15,1.595381e+15,"(0, 3)","(0, 0)","(0, 0)",378613.0,11.0,SZE,1802.0,3262.0,zs_96_08,6282.0,2.0
7057,732,1024968.0,1595381461322220,5361836,1.0,500.0,54700.0,904200.0,0.0,93100730,0,0,0,2000732,7.320103e+10,20200722,1.595381e+15,1.595381e+15,"(0, 4)","(0, 54700)","(0, 500)",809700.0,11.0,SZE,1832.0,3273.0,zs_96_08,6282.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115751,2195,1516277.0,1595554324821118,7851351,1.0,36100.0,30900.0,4919750.0,0.0,93203820,0,0,0,2002195,2.195015e+11,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",4610250.0,11.0,SZE,2337.0,3373.0,zs_96_08,6282.0,2.0
137554,300133,986318.0,1595554260794246,5597896,1.0,4900.0,76400.0,547274.0,0.0,93059800,0,0,0,2300133,3.001330e+13,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",532274.0,11.0,SZE,3678.0,4040.0,zs_96_08,6282.0,2.0
137555,300133,986318.0,1595554260895607,5599887,1.0,14700.0,76400.0,547274.0,0.0,93059900,0,0,0,2300133,3.001330e+13,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",532274.0,11.0,SZE,3678.0,2070.0,zs_94_04,6237.0,2.0
137673,300166,1785467.0,1595554354443268,8758965,2.0,2800.0,155600.0,2256251.0,0.0,93233450,0,0,0,2300166,3.001660e+13,20200724,1.595554e+15,1.595554e+15,"(0, 4, 4, 4)","(0, 155600, 155600, 155600)","(0, 2400, 100, 300)",2246651.0,11.0,SZE,3690.0,4044.0,zs_96_08,6282.0,2.0


In [85]:
checkLog[(checkLog['date'] == 20200724) & (checkLog['accCode'] == 6237)]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,agg,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,start_time,end_time,statusLs,TradePriceLs,TradeQtyLs,vai,ars,exchange,group,order,colo,accCode,time_diff,colo1,colo_broker
94028,12,1019023.0,1595554243158636,5017742,1.0,800.0,59400.0,1248632.0,0.0,93042160,0,0,0,2000012,1.201020e+09,20200724,1.595554e+15,1.595554e+15,"(0, 4, 4)","(0, 59400, 59400)","(0, 700, 100)",1247832.0,11.0,SZE,1093.0,726.0,zs_94_04,6237.0,67750.0,zs,zs_62
94080,12,1701039.0,1595554339117543,8314532,1.0,1900.0,60000.0,2343379.0,0.0,93218120,0,0,0,2000012,1.201701e+09,20200724,1.595554e+15,1.595554e+15,"(0, 4)","(0, 60000)","(0, 1900)",2341479.0,11.0,SZE,1097.0,728.0,zs_94_04,6237.0,3080.0,zs,zs_62
95909,401,1417334.0,1595554322892245,7793340,1.0,1400.0,191000.0,1045169.0,1.0,93201900,0,0,0,2000401,4.010142e+10,20200724,1.595554e+15,1.595554e+15,"(0, 4)","(0, 191000)","(0, 1400)",1043569.0,11.0,SZE,1238.0,810.0,zs_94_04,6237.0,300361.0,zs,zs_62
97187,513,1660850.0,1595554350166811,8637098,2.0,800.0,540000.0,707970.0,0.0,93229170,0,0,0,2000513,5.130166e+10,20200724,1.595554e+15,1.595554e+15,"(0, 4, 4)","(0, 540000, 540000)","(0, 300, 500)",707070.0,11.0,SZE,1300.0,846.0,zs_94_04,6237.0,404153.0,zs,zs_62
99081,623,1215876.0,1595554280126619,6315522,2.0,4100.0,173200.0,346054.0,0.0,93119130,0,0,0,2000623,6.230122e+10,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",323454.0,11.0,SZE,1405.0,910.0,zs_94_04,6237.0,494082.0,zs,zs_62
99222,681,1261494.0,1595554276963282,6194658,2.0,1100.0,175300.0,310506.0,0.0,93115970,0,0,0,2000681,6.810127e+10,20200724,1.595554e+15,1.595554e+15,"(0, 4, 4, 4, 4, 4)","(0, 175300, 175300, 175300, 175300, 175300)","(0, 300, 300, 200, 100, 200)",309306.0,11.0,SZE,1424.0,924.0,zs_94_04,6237.0,590013.0,zs,zs_62
99699,690,1102880.0,1595554277022990,6197265,2.0,11400.0,62100.0,825155.0,0.0,93116030,0,0,0,2000690,6.900111e+10,20200724,1.595554e+15,1.595554e+15,"(0, 4)","(0, 62100)","(0, 11400)",812955.0,11.0,SZE,1451.0,936.0,zs_94_04,6237.0,412727.0,zs,zs_62
100828,739,1516100.0,1595554344016237,8456306,2.0,1300.0,242500.0,636900.0,0.0,93223020,0,0,0,2000739,7.390152e+10,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",635100.0,11.0,SZE,1539.0,968.0,zs_94_04,6237.0,266455.0,zs,zs_62
102414,826,1514598.0,1595554324426094,7838638,1.0,1900.0,84700.0,590886.0,0.0,93203430,0,0,0,2000826,8.260152e+10,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",555886.0,11.0,SZE,1607.0,996.0,zs_94_04,6237.0,95875.0,zs,zs_62
108787,2030,950600.0,1595554255445187,5414091,2.0,600.0,355100.0,2456402.0,1.0,93054450,0,0,0,2002030,2.030010e+11,20200724,1.595554e+15,1.595554e+15,"(0, 4)","(0, 355100)","(0, 600)",2455802.0,11.0,SZE,1937.0,1164.0,zs_94_04,6237.0,88033.0,zs,zs_62


In [80]:
checkLog['date'].unique()

array([20200722, 20200723, 20200724], dtype=int64)

In [79]:
checkLog.groupby(['date', 'colo', 'accCode'])['TransactTime'].describe().astype(int)

count       mean       std       min       25%  \
date     colo     accCode                                                   
20200722 zs_94_04 6237.0     444   95811913   4197220  93000320  93347590   
         zs_96_08 6282.0     483   97332702   5437500  93000200  93114700   
20200723 zs_94_04 6237.0     729   98033955   8632017  93011060  93420980   
         zs_96_08 6282.0     668  100016575  12090076  93000140  93153415   
20200724 zs_94_04 6237.0     367   98214306  11235650  93008060  93330350   
         zs_96_08 6282.0     270   93151413     82013  93008660  93108372   

                                50%        75%        max  
date     colo     accCode                                  
20200722 zs_94_04 6237.0   93811870   95453162  110613500  
         zs_96_08 6282.0   93317170  101949945  110646420  
20200723 zs_94_04 6237.0   94022920  101104900  143139690  
         zs_96_08 6282.0   93714160  102105995  145631850  
20200724 zs_94_04 6237.0   93637060   94639275  145339940  
         zs_96_08 6282.0   93129195   93212092   93318270

In [65]:
checkLog[checkLog.duplicated(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 
                                      'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,agg,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,start_time,end_time,statusLs,TradePriceLs,TradeQtyLs,vai,ars,exchange,group,order,colo,accCode
0,9,1225416.0,1595381490808974,6353082,2.0,1900.0,103000.0,4220341.0,1.0,93130220,0,0,0,2000009,9.012254e+08,20200722,1.595381e+15,1.595381e+15,"(0, 4)","(0, 103000)","(0, 1900)",4218441.0,11.0,SZE,1384.0,1028.0,zs_94_04,6237.0
2,9,1225416.0,1595381490969979,6356974,2.0,1900.0,103000.0,4222841.0,0.0,93130380,0,0,0,2000009,9.012263e+08,20200722,1.595381e+15,1.595381e+15,"(0, 4)","(0, 103000)","(0, 1900)",4218441.0,11.0,SZE,1384.0,1028.0,zs_94_04,6237.0
19,9,1243489.0,1595381493526518,6438804,2.0,1300.0,103100.0,4268341.0,1.0,93132930,0,0,0,2000009,9.012435e+08,20200722,1.595381e+15,1.595381e+15,"(0, 4)","(0, 103100)","(0, 1300)",4267041.0,11.0,SZE,1385.0,1029.0,zs_94_04,6237.0
21,9,1243489.0,1595381493802576,6450624,2.0,1300.0,103100.0,4269641.0,1.0,93133210,0,0,0,2000009,9.012452e+08,20200722,1.595381e+15,1.595381e+15,"(0, 4)","(0, 103100)","(0, 1300)",4267041.0,11.0,SZE,1385.0,1029.0,zs_94_04,6237.0
370,21,1145671.0,1595381469767288,5657154,1.0,100.0,241000.0,486900.0,0.0,93109170,0,0,0,2000021,2.101146e+09,20200722,1.595381e+15,1.595381e+15,"(0, 3)","(0, 0)","(0, 0)",482400.0,11.0,SZE,1400.0,3120.0,zs_96_08,6282.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144891,300630,1314258.0,1595554306355717,7257516,1.0,400.0,486600.0,181742.0,0.0,93145360,0,0,0,2300630,3.006300e+13,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",181342.0,11.0,SZE,4119.0,4238.0,zs_96_08,6282.0
144892,300630,1314258.0,1595554306355718,7257517,1.0,400.0,486600.0,181742.0,0.0,93145360,0,0,0,2300630,3.006300e+13,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",181342.0,11.0,SZE,4119.0,4238.0,zs_96_08,6282.0
144893,300630,1314258.0,1595554306358553,7257580,1.0,400.0,486600.0,181742.0,0.0,93145360,0,0,0,2300630,3.006300e+13,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",181342.0,11.0,SZE,4119.0,4238.0,zs_96_08,6282.0
144896,300630,1314258.0,1595554306364950,7257739,1.0,400.0,486600.0,181742.0,0.0,93145370,0,0,0,2300630,3.006300e+13,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",181342.0,11.0,SZE,4119.0,4238.0,zs_96_08,6282.0


In [63]:
checkLog

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,agg,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,start_time,end_time,statusLs,TradePriceLs,TradeQtyLs,vai,ars,exchange,group,order,colo,accCode,time_diff,colo1,colo_broker
52,9,2307546.0,1595381690167743,11926646,2.0,37200.0,102700.0,9778269.0,0.0,93449570,0,0,0,2000009,9.023083e+08,20200722,1.595382e+15,1.595382e+15,"(0, 3)","(0, 0)","(0, 0)",9730827.0,11.0,SZE,1386.0,1030.0,zs_94_04,6237.0,151934.0,zs,zs_62
100,9,2633953.0,1595381767036980,13746180,2.0,700.0,102700.0,10724305.0,1.0,93606440,0,0,0,2000009,9.026363e+08,20200722,1.595382e+15,1.595382e+15,"(0, 4)","(0, 102700)","(0, 700)",10695905.0,11.0,SZE,1389.0,1033.0,zs_94_04,6237.0,528798.0,zs,zs_62
138,9,2758907.0,1595381792712506,14396709,2.0,4400.0,102500.0,11189560.0,1.0,93632120,0,0,0,2000009,9.027598e+08,20200722,1.595382e+15,1.595382e+15,"(0, 4)","(0, 102500)","(0, 4400)",11180969.0,11.0,SZE,1390.0,1034.0,zs_94_04,6237.0,178214.0,zs,zs_62
150,9,2761298.0,1595381793164805,14404406,2.0,2900.0,102500.0,11396269.0,1.0,93632570,0,0,0,2000009,9.027618e+08,20200722,1.595382e+15,1.595382e+15,"(0, 4, 4, 4)","(0, 102500, 102500, 102500)","(0, 9, 2000, 891)",11206060.0,11.0,SZE,1391.0,1035.0,zs_94_04,6237.0,126332.0,zs,zs_62
163,9,2771932.0,1595381795178699,14458855,2.0,5800.0,102400.0,11487869.0,1.0,93634580,0,0,0,2000009,9.027722e+08,20200722,1.595382e+15,1.595382e+15,"(0, 4)","(0, 102400)","(0, 5800)",11470569.0,11.0,SZE,1392.0,1036.0,zs_94_04,6237.0,52344.0,zs,zs_62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143767,300558,1847043.0,1595554386227329,9663218,1.0,600.0,1460800.0,324200.0,0.0,93305230,0,0,0,2300558,3.005580e+13,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",322000.0,11.0,SZE,4062.0,4214.0,zs_96_08,6282.0,401937.0,zs,zs_62
144166,300595,2348118.0,1595554513354185,13287282,1.0,100.0,639400.0,626024.0,1.0,93512360,0,0,0,2300595,3.005950e+13,20200724,1.595555e+15,1.595555e+15,"(0, 4)","(0, 639400)","(0, 100)",625724.0,11.0,SZE,4084.0,2303.0,zs_94_04,6237.0,36580.0,zs,zs_62
144203,300595,4189434.0,1595554968752390,24319498,1.0,100.0,630100.0,1502792.0,1.0,94247750,0,0,0,2300595,3.005950e+13,20200724,1.595555e+15,1.595555e+15,"(0, 4)","(0, 630100)","(0, 100)",1502692.0,11.0,SZE,4089.0,2308.0,zs_94_04,6237.0,140477.0,zs,zs_62
144717,300618,1118994.0,1595554266628371,5807919,2.0,100.0,661200.0,428100.0,0.0,93105630,0,0,0,2300618,3.006180e+13,20200724,1.595554e+15,1.595554e+15,"(0, 3)","(0, 0)","(0, 0)",427900.0,11.0,SZE,4108.0,4236.0,zs_96_08,6282.0,63185.0,zs,zs_62


In [48]:
orderLog['updateType'].unique()

array([0., 1.])